## Imports

In [8]:
pip list

Package                  Version
------------------------ -------------------
absl-py                  0.15.0
ale-py                   0.7.1
apturl                   0.5.2
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
asn1crypto               0.24.0
astor                    0.8.1
astunparse               1.6.3
async-generator          1.10
attrs                    21.4.0
backcall                 0.2.0
bleach                   4.1.0
Box2D                    2.3.10
Box2D-kengz              2.3.3
Brlapi                   0.6.6
cached-property          1.5.2
cachetools               4.2.4
certifi                  2021.10.8
cffi                     1.15.0
chardet                  3.0.4
charset-normalizer       2.0.10
clang                    5.0
click                    6.7
cloudpickle              1.6.0
colorama                 0.3.7
command-not-found        0.3
cryptography             2.1.4
cupshelpers              1.0
cycler                   0.11.0
daiquiri           

2022-02-09 16:15:35,092 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...


Note: you may need to restart the kernel to use updated packages.


2022-02-09 16:15:36,094 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:37,097 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:38,098 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:39,100 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:40,101 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:41,102 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:42,104 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:43,105 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:44,107 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:45,108 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:46,109 [15351] INFO    

2022-02-09 16:17:02,219 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:17:03,220 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:17:04,222 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:17:05,223 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:17:06,225 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:17:07,226 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:17:08,227 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:17:09,229 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:17:10,230 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:17:11,232 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:17:12,233 [15351] INFO    

In [ ]:
pip uninstall gym -y

In [ ]:
pip install gym

In [ ]:
pip uninstall tensorflow -y

In [ ]:
pip install tensorflow==1.15.2

In [ ]:
from tensorflow.python.compiler import tensorrt as trt

In [ ]:
import stable_baselines
stable_baselines.__version__

In [ ]:
pip uninstall gym -y

In [ ]:
pip install gym==0.20.0

In [1]:
import os
import gym
import time
import ml_monitor
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines import DDPG, TD3
from stable_baselines.ddpg.policies import LnMlpPolicy
from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common.noise import AdaptiveParamNoiseSpec, NormalActionNoise
from stable_baselines.common.callbacks import BaseCallback

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/home/darias/.local/lib/python3.6/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


## Gym environment

In [ ]:
env = gym.make('LunarLanderContinuous-v2')

In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

In [ ]:
env.close()

## Training phase

### Callback function

In [2]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """
    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model')
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        my_monitor.monitor("testing_switch",0)
        my_monitor.monitor("training_switch",1)
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), 'timesteps')
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print("Num timesteps: {}".format(self.num_timesteps))
                    print("Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(self.best_mean_reward, mean_reward))
                    my_monitor.monitor("num_timesteps",self.num_timesteps)
                    my_monitor.monitor("mean_reward",mean_reward)
                    
            # New best model
            if mean_reward > self.best_mean_reward:
                self.best_mean_reward = mean_reward
                my_monitor.monitor("best_mean_reward",self.best_mean_reward)
                # Saving best model
                if self.verbose > 0:
                    print("Saving new best model to {}".format(self.save_path))
                self.model.save(self.save_path)

        return True

In [3]:
# Create log dir
log_dir = "machine-learning/logs/"
os.makedirs(log_dir, exist_ok=True)

In [4]:
# Creating the environment
env = gym.make('LunarLanderContinuous-v2')
# Logs will be saved in log_dir/monitor.csv
env = Monitor(env, log_dir)

### Training the model

In [5]:
# Add some param noise for exploration
param_noise = AdaptiveParamNoiseSpec(initial_stddev=0.1, desired_action_stddev=0.1)
# Create the callback: check every 1000 steps
callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)

In [6]:
my_monitor = ml_monitor.Monitor()
my_monitor.start()

2022-02-09 14:23:35,189 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 14:23:36,190 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...


In [7]:
# Because we use parameter noise, we should use a MlpPolicy with layer normalization
model = DDPG(LnMlpPolicy, env, param_noise=param_noise, verbose=False)
# Train the agent
model.learn(total_timesteps=int(2e6), callback=callback)

2022-02-09 14:23:36,959 [15351] WARNING  tensorflow: From /home/darias/.local/lib/python3.6/site-packages/stable_baselines/common/tf_util.py:191: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



2022-02-09 14:23:36,961 [15351] WARNING  tensorflow: From /home/darias/.local/lib/python3.6/site-packages/stable_baselines/common/tf_util.py:200: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



2022-02-09 14:23:36,966 [15351] WARNING  tensorflow: From /home/darias/.local/lib/python3.6/site-packages/stable_baselines/ddpg/ddpg.py:332: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



2022-02-09 14:23:36,967 [15351] WARNING  tensorflow: From /home/darias/.local/lib/python3.6/site-packages/stable_baselines/common/input.py:25: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



Instructions for updating:
Use keras.layers.flatten instead.


...

2022-02-09 16:15:26,075 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:27,077 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:28,080 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...


Num timesteps: 1999000
Best mean reward: 280.91 - Last mean reward per episode: 277.73


2022-02-09 16:15:29,081 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:30,083 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:31,086 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...


Num timesteps: 2000000
Best mean reward: 280.91 - Last mean reward per episode: 276.18


2022-02-09 16:15:32,088 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:33,089 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-09 16:15:34,090 [15351] INFO     ml_monitor.logging: Starting metrics logging thread...


In [ ]:
model.save("machine-learning/models/LunarLander2e6")

## Testing phase

### Fast import

In [ ]:
pip install gym==0.20.0

In [ ]:
import os
import gym
import time
import numpy as np
import matplotlib.pyplot as plt
import ml_monitor

from stable_baselines import DDPG, TD3
from stable_baselines.ddpg.policies import LnMlpPolicy
from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common.noise import AdaptiveParamNoiseSpec, NormalActionNoise
from stable_baselines.common.callbacks import BaseCallback

### Loading model

In [ ]:
del model

In [ ]:
env = gym.make('LunarLanderContinuous-v2')
model = DDPG.load("machine-learning/models/LunarLander1e6")

### Agent environment

In [ ]:
my_monitor = ml_monitor.Monitor()
my_monitor.start()

In [ ]:
my_monitor.monitor("testing_switch",1)
my_monitor.monitor("training_switch",0)
episodes = 100
for episodes in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    my_monitor.monitor("testing_episodes",episodes)
    my_monitor.monitor("testing_score",score)
    print('Episode:{} Score:{}'.format(episodes, score))
    time.sleep(3)
env.close()

In [ ]:
env.close()

## Ploting results

In [ ]:
from stable_baselines import results_plotter

# Helper from the library
results_plotter.plot_results([log_dir], 1e5, results_plotter.X_TIMESTEPS, "DDPG LunarLander")

In [ ]:
def moving_average(values, window):
    """
    Smooth values by doing a moving average
    :param values: (numpy array)
    :param window: (int)
    :return: (numpy array)
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, 'valid')


def plot_results(log_folder, title='Learning Curve'):
    """
    plot the results

    :param log_folder: (str) the save location of the results to plot
    :param title: (str) the title of the task to plot
    """
    x, y = ts2xy(load_results(log_folder), 'timesteps')
    y = moving_average(y, window=50)
    # Truncate x
    x = x[len(x) - len(y):]

    fig = plt.figure(title)
    plt.plot(x, y)
    plt.xlabel('Number of Timesteps')
    plt.ylabel('Rewards')
    plt.title(title + " Smoothed")
    plt.show()


In [ ]:
plot_results(log_dir)